In [1]:
import comet_ml
from comet_ml import Experiment
from sklearn.model_selection import train_test_split
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [2]:
   
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="D4ClOaV8GaZLHo0I2et7bOHgy",
                        project_name="ann-combustion-test", workspace="mdcabrera"
                       ,auto_param_logging=False)




COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mdcabrera/ann-combustion-test/ae3217d5560d4de38fdcf1f16cefba00



In [3]:
samples=5000
step=1e-7
batch_size = 128
epochs = 20
optimizer = 'sgd'
architecture = [[7,'tanh'],[7,'linear']]
loss_function='mean_squared_error'
min_delta=1e-8

In [4]:
path='../data/'+str(samples)+'_'+str(step)+'/'
i=1
Input=pd.read_csv(path+'input/input_'+str(i)+'.csv', sep=',')
input_var=Input.columns[:8]
Input=Input[input_var].values

Output=pd.read_csv(path+'output/output_'+str(i)+'.csv', sep=',')
Output_var=Output.columns[1:8]
Output=Output[Output_var].values

In [5]:
# Splitting the dataset into train, test and val

X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(Input, Output, test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)
s=X_train.shape[1]

(1460307, 8) (312923, 8) (312923, 8) (1460307, 7) (312923, 7) (312923, 7)


In [6]:
model=Sequential()
model.add(InputLayer(input_shape=(s,)))
for x in architecture:
    model.add(Dense(x[0],activation=x[1]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 7)                 63        
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 56        
Total params: 119
Trainable params: 119
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

In [8]:
params={'batch_size':batch_size,
        'epochs':epochs,
        'architecture':architecture,
        'optimizer':optimizer,
        'loss_function':loss_function,
        'samples':samples,
        'step_size':step,
        'min_delta':min_delta
}


In [9]:
with experiment.train():
    history = model.fit(X_train, Y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(X_test, Y_test),
                        callbacks=[EarlyStopping(monitor='val_loss', min_delta=min_delta,patience=3, verbose=1, mode='auto')])



Train on 1460307 samples, validate on 312923 samples
Epoch 1/20
    128/1460307 [..............................] - ETA: 1:56:27 - loss: 1.2819 - accuracy: 0.0000e+00

COMET INFO: Ignoring automatic log_metric('batch_batch') because 'keras:batch_batch' is in COMET_LOGGING_METRICS_IGNORE
COMET INFO: Ignoring automatic log_metric('batch_size') because 'keras:batch_size' is in COMET_LOGGING_METRICS_IGNORE


1460307/1460307 [==============================] - 15s 10us/step - loss: 0.0037 - accuracy: 0.9968 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 2/20
1460307/1460307 [==============================] - 14s 10us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 3/20
1460307/1460307 [==============================] - 16s 11us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 4/20
1460307/1460307 [==============================] - 15s 10us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 5/20
1460307/1460307 [==============================] - 16s 11us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 00005: early stopping


In [10]:
with experiment.test():
    loss, accuracy = model.evaluate(X_val, Y_val)
    metrics = {
        'loss':loss,
        'accuracy':accuracy
    }
    experiment.log_metrics(metrics)

experiment.log_parameters(params)
experiment.log_dataset_hash(X_train)

312923/312923 [==============================] - 6s 18us/step


In [11]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/mdcabrera/ann-combustion-test/ae3217d5560d4de38fdcf1f16cefba00
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     test_accuracy                  : 1.0
COMET INFO:     test_loss                      : 0.0011761546594200763
COMET INFO:     train_accuracy [5]             : (0.996844470500946, 1.0)
COMET INFO:     train_batch_accuracy [5705]    : (0.0, 1.0)
COMET INFO:     train_batch_loss [5705]        : (0.00038236600812524557, 1.2819194793701172)
COMET INFO:     train_epoch_duration [5]       : (14.414001476000067, 15.708740182999918)
COMET INFO:     train_loss [5]                 : (0.001172803632927275, 0.0036618122536835815)
COMET INFO:     train_val_accuracy [5]         : 1.0
COMET INFO:     train_val_loss [5]             : (0.0011742357

In [12]:
import numpy as np
import cantera as ct
import pandas as pd
import scipy
from scipy import integrate

#Reactor solver
class ReactorOde(object):
    def __init__(self, gas):
        # Parameters of the ODE system and auxiliary data are stored in the
        # ReactorOde object.
        self.gas = gas
        self.P = gas.P

    def __call__(self, t, y):
        """the ODE function, y' = f(t,y) """

        # State vector is [T, Y_1, Y_2, ... Y_K]
        self.gas.set_unnormalized_mass_fractions(y[1:])
        self.gas.TP = y[0], self.P
        rho = self.gas.density

        wdot = self.gas.net_production_rates
        dTdt = - (np.dot(self.gas.partial_molar_enthalpies, wdot) /
                  (rho * self.gas.cp))
        dYdt = wdot * self.gas.molecular_weights / rho

        return np.hstack((dTdt, dYdt))

T=X_val[0][0]
YInit=X_val[0][1:]

#choose the wanted mechanism here
gas = ct.Solution('../Mechanisms/WD_Laubscher.cti')
# Initial condition
P = ct.one_atm

gas.TPY = T, P, YInit
y0 = np.hstack((gas.T, gas.Y))

# Set up objects representing the ODE and the solver
ode = ReactorOde(gas)
solver = scipy.integrate.ode(ode)
solver.set_integrator('vode', method='bdf', with_jacobian=True)
solver.set_initial_value(y0, 0.0)

# Integrate the equations, keeping T(t) and Y(k,t)
#this time should be varied, it corresponds to the residence time (tau_star) in EDC
t_end = 1e-4
states = ct.SolutionArray(gas, 1, extra={'t': [0.0]})
dt = 1e-7
#solving the equations
while solver.successful() and solver.t < t_end:
    solver.integrate(solver.t + dt)
    gas.TPY = solver.y[0], P, solver.y[1:]
    states.append(gas.state, t=solver.t)


dfy=pd.DataFrame(states.Y,columns=gas.species_names)
dfy['Temp']=states.T
dfy.head()

,CH4,O2,H2O,N2,CO,CO2,H2,Temp
0,0.024682,0.227246,4.954568e-12,0.748072,1.280604e-12,1.795527e-25,4.698226e-07,612.751804
1,0.024682,0.227246,4.962064e-12,0.748072,1.282541e-12,1.796890e-25,4.698226e-07,612.751804
2,0.024682,0.227246,4.969559e-12,0.748072,1.284479e-12,1.798257e-25,4.698226e-07,612.751804
3,0.024682,0.227246,4.977055e-12,0.748072,1.286416e-12,1.799624e-25,4.698226e-07,612.751804
4,0.024682,0.227246,4.984550e-12,0.748072,1.288354e-12,1.801015e-25,4.698226e-07,612.751804


In [13]:
states.T
K=X_val[0].reshape(1,8)
Z=np.copy(K)
K2=np.copy(K)
K2[0][0]=states.T[0]


RT=np.copy(K2)
for j in range (1,len(states.t)):
    #break
    L=model.predict(Z)
    Z=np.copy(np.append(states.T[j],L))
    Z=Z.reshape(1,8)
    
    
    K2=np.copy(np.append(states.T[j],L))
    K2=K2.reshape(1,8)
    
    #Z=Z.reshape(1,8)
    #K2=K2.reshape(1,8)
    RT=np.vstack((RT,K2))
    #states2.T

dfx=pd.DataFrame(RT,columns=(['Temp']+gas.species_names))
dfx.head()


,Temp,CH4,O2,H2O,N2,CO,CO2,H2
0,612.751804,0.024682,0.227246,4.954566e-12,0.748072,1.280604e-12,1.795526e-25,4.698224e-07
1,612.751804,0.030561,0.197927,1.300529e-02,0.737414,2.546746e-03,1.798633e-02,5.595856e-04
2,612.751804,0.030561,0.197927,1.300529e-02,0.737414,2.546746e-03,1.798633e-02,5.595856e-04
3,612.751804,0.030561,0.197927,1.300529e-02,0.737414,2.546746e-03,1.798633e-02,5.595856e-04
4,612.751804,0.030561,0.197927,1.300529e-02,0.737414,2.546746e-03,1.798633e-02,5.595856e-04
